In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import time
import requests
import io
from scipy.stats import linregress

In [2]:
# ==========================================
# 1. HELPER FUNCTIONS (The Math & Logic)
# ==========================================
def get_interest_coverage(financials):
    """Calculates Interest Coverage Ratio (Solvency Check)"""
    try:
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan

        if 'Interest Expense' in financials.index:
            interest = financials.loc['Interest Expense'].iloc[0]
        elif 'Interest Expense Non Operating' in financials.index:
            interest = financials.loc['Interest Expense Non Operating'].iloc[0]
        else:
            return 100.0 # Debt-free assumption

        interest = abs(interest)
        if interest == 0: return 100.0
            
        return round(ebit / interest, 2)
    except:
        return np.nan

def calculate_roic(financials, balance_sheet):
    """Calculates Return on Invested Capital (Quality Check)"""
    try:
        # NOPAT
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan
            
        tax_provision = financials.loc['Tax Provision'].iloc[0] if 'Tax Provision' in financials.index else 0
        pre_tax_income = financials.loc['Pretax Income'].iloc[0] if 'Pretax Income' in financials.index else 0
        
        if pre_tax_income > 0:
            tax_rate = max(0.0, min(tax_provision / pre_tax_income, 0.30))
        else:
            tax_rate = 0.21
            
        nopat = ebit * (1 - tax_rate)

        # Invested Capital
        total_equity = balance_sheet.loc['Stockholders Equity'].iloc[0]
        
        total_debt = 0
        if 'Total Debt' in balance_sheet.index:
            total_debt = balance_sheet.loc['Total Debt'].iloc[0]
        elif 'Long Term Debt' in balance_sheet.index:
            total_debt = balance_sheet.loc['Long Term Debt'].iloc[0]
             
        cash = balance_sheet.loc['Cash And Cash Equivalents'].iloc[0]
        
        invested_capital = total_equity + total_debt - cash
        if invested_capital <= 0: return np.nan
            
        return round((nopat / invested_capital) * 100, 2)
    except:
        return np.nan

def calculate_z_score(info, financials, balance_sheet):
    """Calculates Altman Z-Score"""
    try:
        total_assets = balance_sheet.loc['Total Assets'].iloc[0]
        # Fallback for liabilities
        if 'Total Liabilities Net Minority Interest' in balance_sheet.index:
            total_liab = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
        elif 'Total Liabilities' in balance_sheet.index:
            total_liab = balance_sheet.loc['Total Liabilities'].iloc[0]
        else:
            return np.nan
            
        current_assets = balance_sheet.loc['Current Assets'].iloc[0]
        current_liab = balance_sheet.loc['Current Liabilities'].iloc[0]
        
        working_capital = current_assets - current_liab
        retained_earnings = balance_sheet.loc['Retained Earnings'].iloc[0] if 'Retained Earnings' in balance_sheet.index else 0
        
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan
            
        market_cap = info.get('marketCap', 0)
        sales = financials.loc['Total Revenue'].iloc[0]

        # Ratios
        A = working_capital / total_assets
        B = retained_earnings / total_assets
        C = ebit / total_assets
        D = market_cap / total_liab
        E = sales / total_assets

        z_score = (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
        return round(z_score, 2)
    except:
        return np.nan

def get_margin_trend(financials):
    """Calculates 4-Year Gross Margin Slope"""
    try:
        limit = min(4, len(financials.columns))
        years = financials.columns[:limit]
        
        margins = []
        for date in years:
            rev = financials.loc['Total Revenue'][date]
            profit = financials.loc['Gross Profit'][date]
            if rev == 0 or np.isnan(rev): margins.append(0)
            else: margins.append(profit/rev)
            
        if len(margins) < 2: return "N/A"
        
        margins = margins[::-1] # Flip to Chronological
        slope, _, _, _, _ = linregress(range(len(margins)), margins)
        
        if slope > 0.005: return "Improving"
        elif slope < -0.005: return "Deteriorating"
        else: return "Stable"
    except:
        return "N/A"

In [3]:
# ==========================================
# 2. STEP 1: FETCH UNIVERSE (FIXED 403 ERROR)
# ==========================================
def get_universe_from_indices():
    print("--- STEP 1: Fetching Universe from Indices (Wikipedia) ---")
    all_tickers = []
    
    # Header to mimic a browser and bypass Wikipedia's block
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # --- 1. USA: S&P 500 & 400 ---
    us_indices = [
        ("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies", "USA - Large Cap"),
        ("https://en.wikipedia.org/wiki/List_of_S%26P_400_companies", "USA - Mid Cap")
    ]
    
    for url, label in us_indices:
        print(f"   Fetching {label}...")
        try:
            # Request content with headers first
            r = requests.get(url, headers=headers)
            r.raise_for_status()
            
            # Read HTML from the response text
            tables = pd.read_html(io.StringIO(r.text))
            df = tables[0]
            
            # Standardize columns
            if 'Symbol' in df.columns: df.rename(columns={'Symbol': 'Ticker'}, inplace=True)
            if 'Security' in df.columns: df.rename(columns={'Security': 'Company'}, inplace=True)
            if 'GICS Sector' in df.columns: df.rename(columns={'GICS Sector': 'Sector'}, inplace=True)
            
            # Clean Tickers (BF.B -> BF-B)
            df['Ticker'] = df['Ticker'].str.replace('.', '-', regex=False)
            df = df[['Ticker', 'Company', 'Sector']]
            
            all_tickers.append(df)
            print(f"      -> Added {len(df)} stocks.")
            
        except Exception as e:
            print(f"      Error fetching {label}: {e}")

    # --- 2. CANADA: TSX Composite ---
    print(f"   Fetching Canada (TSX Composite)...")
    try:
        url_ca = "https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index"
        r_ca = requests.get(url_ca, headers=headers)
        r_ca.raise_for_status()
        
        tables = pd.read_html(io.StringIO(r_ca.text))
        
        # Find the correct table for TSX
        df_ca = pd.DataFrame()
        for t in tables:
            if 'Symbol' in t.columns:
                df_ca = t
                break
        
        if not df_ca.empty:
            df_ca.rename(columns={'Symbol': 'Ticker', 'Company': 'Company', 'Sector': 'Sector'}, inplace=True)
            
            # Add .TO suffix
            df_ca['Ticker'] = df_ca['Ticker'].apply(lambda x: x + ".TO" if "." not in x else x.replace('.', '-') + ".TO")
            
            # Select only needed columns (Handle missing Sector column if Wikipedia changes)
            cols = ['Ticker', 'Company', 'Sector'] if 'Sector' in df_ca.columns else ['Ticker', 'Company']
            df_ca = df_ca[cols]
            
            all_tickers.append(df_ca)
            print(f"      -> Added {len(df_ca)} stocks.")
            
    except Exception as e:
        print(f"      Error fetching Canada: {e}")

    if not all_tickers:
        return pd.DataFrame()
        
    df_final = pd.concat(all_tickers, ignore_index=True)
    df_final.drop_duplicates(subset=['Ticker'], inplace=True)
    
    print(f"   Success! Combined Universe: {len(df_final)} stocks.")
    return df_final

In [ ]:


# ==========================================
# 3. STEP 2 & 3: EXECUTION ENGINE
# ==========================================
def run_screener_split():
    # 1. Get the List
    candidates = get_universe_from_indices()
    
    if candidates.empty:
        print("No stocks found to analyze.")
        return None, None, None, None, None

    print(f"\n--- STEP 2: Running Credit Risk Model on {len(candidates)} Stocks ---\n")
    
    fortress_data = []
    moonshot_data = []
    distress_data = []
    financial_data = [] 
    middle_data = []    
    
    # Iterate
    for index, row in candidates.iterrows():
        ticker = row['Ticker']
        
        # Polite throttling
        if index % 10 == 0: 
            print(f"   Processed {index} / {len(candidates)}...", end='\r')
            # time.sleep(0.1) # Removed sleep for speed, add back if you get errors
        
        try:
            stock = yf.Ticker(ticker)
            
            # --- 1. SECTOR FILTER ---
            info = stock.info
            sector = info.get('sector', row.get('Sector', 'Unknown'))
            
            # Skip Financials (Banks break Z-Score)
            if 'Financial' in str(sector):
                financial_data.append({'Ticker': ticker, 'Sector': sector})
                continue

            # --- 2. FETCH PRICE (CRITICAL) ---
            current_price = info.get('currentPrice', info.get('regularMarketPreviousClose', 0))
            if current_price == 0: continue

            # --- 3. FETCH DEEP DATA ---
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty: continue
            
            # --- 4. RUN METRICS ---
            z_score = calculate_z_score(info, fin, bs)
            trend = get_margin_trend(fin)
            int_cov = get_interest_coverage(fin)
            roic = calculate_roic(fin, bs)
            
            # Packet
            stock_data = {
                'Ticker': ticker,
                'Company': row.get('Company', 'N/A'),
                'Z-Score': z_score,
                'Price': current_price,
                'Margin_Trend': trend,
                'Int_Cov': int_cov,
                'ROIC': roic,
                'Sector': sector
            }
            
            # --- 5. SORT ---
            if (z_score > 2.99) and (trend in ["Improving", "Stable"]) and (int_cov > 4.0) and (roic > 5.0):
                fortress_data.append(stock_data)
            elif (z_score < 1.8) or (int_cov < 1.5):
                distress_data.append(stock_data)
            elif (z_score < 2.5) and (trend == "Improving"):
                moonshot_data.append(stock_data)
            else:
                middle_data.append(stock_data)
                
        except Exception:
            continue

    # Convert to DataFrames
    fortress_df = pd.DataFrame(fortress_data)
    moonshot_df = pd.DataFrame(moonshot_data)
    distress_df = pd.DataFrame(distress_data)
    financial_df = pd.DataFrame(financial_data)
    middle_df = pd.DataFrame(middle_data)
    
    print(f"\n\n--- ANALYSIS COMPLETE ---\n")
    print(f"Fortress (Safe):     {len(fortress_df)} stocks")
    print(f"Moonshot (Risky):    {len(moonshot_df)} stocks")
    print(f"Distress (Avoid):    {len(distress_df)} stocks")
    print(f"Financials (Skip):   {len(financial_df)} stocks")
    
    return fortress_df, moonshot_df, distress_df, financial_df, middle_df

# --- RUN IT ---
fortress_df, moonshot_df, distress_df, fins_df, mid_df = run_screener_split()

# --- DISPLAY ---
if fortress_df is not None and not fortress_df.empty:
    print("\n--- FORTRESS STOCKS (High Quality) ---")
    display(fortress_df.sort_values(by='Z-Score', ascending=False).head(10))

if moonshot_df is not None and not moonshot_df.empty:
    print("\n--- MOONSHOT STOCKS (Speculative) ---")
    display(moonshot_df.sort_values(by='Price', ascending=True).head(10))
    
if distress_df is not None and not distress_df.empty:
    print("\n--- DISTRESS (Avoid / Short Candidates) ---")
    display(distress_df.sort_values(by='Z-Score', ascending=True).head(10))

--- STEP 1: Fetching Universe from Indices (Wikipedia) ---
   Fetching USA - Large Cap...
      -> Added 501 stocks.
   Fetching USA - Mid Cap...
      -> Added 401 stocks.
   Fetching Canada (TSX Composite)...
   Success! Combined Universe: 902 stocks.

--- STEP 2: Running Credit Risk Model on 902 Stocks ---



Exception ignored from cffi callback <function buffer_callback at 0x000001C24D7AF6A0>:
Traceback (most recent call last):
  File "c:\Users\James\anaconda3\Lib\site-packages\curl_cffi\curl.py", line 101, in buffer_callback
    @ffi.def_extern()
KeyboardInterrupt: 


In [ ]:
# ==========================================
# Watchlist Combiner (Finviz + YFinance)
# ==========================================

import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = ['GRND','ARCC','BANC','ONB','UBER','ADMA','MIR','APG','SEI','FLEX','DD'] 

def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")
    
    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []
    
    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()
            
            finviz_data.append({
                'Ticker': ticker,
                'Recom': info.get('Recom', np.nan),
                'Target_Price': info.get('Target Price', np.nan)
            })
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append({'Ticker': ticker, 'Recom': np.nan, 'Target_Price': np.nan})

    df_finviz = pd.DataFrame(finviz_data)
    
    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")
    
    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)
        yf_stats = []
        
        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=['Close'])
                if len(df) < 20: 
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df['Close'].iloc[-1]
                prev_close = df['Close'].iloc[-2]
                
                high_52 = df['High'].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100
                
                change_pct = ((current_price - prev_close) / prev_close) * 100
                
                # Volatility (30-day Std Dev)
                volatility = df['Close'].pct_change().std() * 100
                
                # Relative Volume
                curr_vol = df['Volume'].iloc[-1]
                avg_vol = df['Volume'].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df['Close'].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100 

                yf_stats.append({
                    'Ticker': ticker,
                    'Price': round(current_price, 2),
                    'Change_%': round(change_pct, 2),
                    '52W_MA': round(ma_52w, 2),          # <--- Added Here
                    'Drop_from_High_%': round(drop_from_high, 2),
                    'Volatility_%': round(volatility, 2),
                    'Rel_Volume': round(rel_vol, 2)
                })
                
            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue
                
        df_yf = pd.DataFrame(yf_stats)
        
    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on='Ticker', how='outer')
        else:
            master_df = df_finviz
            
        # Added '52W_MA' to this list so it displays in the final table
        cols = ['Ticker', 'Price', 'Change_%', '52W_MA', 'Drop_from_High_%', 'Recom', 'Target_Price', 'Rel_Volume', 'Volatility_%']
        
        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()

# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if 'Drop_from_High_%' in watchlist_df.columns:
        watchlist_df['Drop_from_High_%'] = pd.to_numeric(watchlist_df['Drop_from_High_%'], errors='coerce')
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by='Drop_from_High_%', ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")